# Features summary
1. Elo rating of each chef at the time of the cook-off
1. Demography difference between the chef and the guest they are cooking for
1. Favorite ingrdients


In [661]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
import pickle
import numpy as np
import pandas as pd
import copy
from collections import Counter

# Step 1: Load dataset
I imported pickle dataframe which I saved on my previous exercise using elo rating to predict the winner. 

In [2]:
with open("../data/game_episode_data.pkl", 'rb') as fp:
    game_data = pickle.load(fp)
    
with open("../data/elo_df.pkl", 'rb') as fp:
    elo_df = pickle.load(fp)
    
game_data = game_data.reset_index()
elo_df = elo_df.reset_index()

In [3]:
game_data = game_data.rename(columns = {"win": "win_name", "lose":"lose_name","date":"date_", "guest":"guest_name"})
game_df = game_data.merge(elo_df)
game_df = game_df[["date","win_name","lose_name","guest_name","w_elo_before_game","l_elo_before_game", "win_epi_num", "lose_epi_num"]]
game_df[:3]

,date,win_name,lose_name,guest_name,w_elo_before_game,l_elo_before_game,win_epi_num,lose_epi_num
0,1/19/2015,최현석,박준우,소유진,1500,1500,-1,-1
1,1/19/2015,홍석천,샘킴,소유진,1500,1500,-1,-1
2,1/26/2015,미카엘,김풍,가희,1500,1500,-1,-1


# Step 2: Add expected win

In [51]:
def expected_result(elo_a, elo_b):
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a

mean_elo = 1500
elo_width = 400
k_factor = 64

def update_elo(winner_elo, loser_elo):
    expected_win = expected_result(winner_elo, loser_elo)
    change_in_elo = k_factor * (1-expected_win)
    winner_elo += change_in_elo
    loser_elo -= change_in_elo
    return winner_elo, loser_elo


In [5]:
elo_a = game_df["w_elo_before_game"].tolist()
elo_b = game_df["l_elo_before_game"].tolist()

expected_win = []
for idx, a in enumerate(elo_a):
    expected_a = expected_result(a, elo_b[idx])
    expected_win.append(expected_a)

In [6]:
game_df["w_win_prob"] = pd.Series(expected_win).values
game_df["l_win_prob"] = 1 - game_df["w_win_prob"]

In [7]:
game_df[-10:]

,date,win_name,lose_name,guest_name,w_elo_before_game,l_elo_before_game,win_epi_num,lose_epi_num,w_win_prob,l_win_prob
303,1/8/2018,미카엘,레이먼 킴,장서희,1406,1645,6568,6567,0.201685,0.798315
304,1/8/2018,오세득,김풍,장서희,1559,1370,6566,6565,0.748000,0.252000
305,1/15/2018,오세득,유현수,데프콘_(가수),1575,1485,6572,6571,0.626699,0.373301
306,1/15/2018,김풍,이연복,데프콘_(가수),1354,1644,6569,6570,0.158508,0.841492
307,1/22/2018,샘킴,정호영,김동현_(격투기_선수),1471,1503,6575,6576,0.454078,0.545922
308,1/22/2018,정지선,레이먼 킴,김동현_(격투기_선수),1500,1594,6574,-1,0.367930,0.632070
309,1/29/2018,유현수,정지선,이계인,1461,1540,6579,6580,0.388229,0.611771
310,1/29/2018,김풍,정호영,이계인,1408,1468,6577,6578,0.414501,0.585499
311,2/5/2018,레이먼 킴,샘킴,김신영,1554,1506,6584,6583,0.568641,0.431359
312,2/5/2018,미카엘,이연복,김신영,1458,1590,6582,6581,0.318678,0.681322


# Step 4: Reshape dataframe

In [8]:
win_table = game_df[["date","win_name","lose_name", "guest_name","win_epi_num","w_win_prob"]]
win_table = win_table.rename(columns = {"win_name": "chef_name","lose_name": "against","win_epi_num":"epi_num","w_win_prob":"win_prob"})
win_table["result"] = 1
win_table = win_table.reset_index()
win_table[:3]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result
0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1
1,1,1/19/2015,홍석천,샘킴,소유진,-1,0.5,1
2,2,1/26/2015,미카엘,김풍,가희,-1,0.5,1


In [9]:
lose_table = game_df[["date","lose_name","win_name", "guest_name","lose_epi_num","l_win_prob"]]
lose_table = lose_table.rename(columns = {"lose_name": "chef_name","win_name": "against","lose_epi_num":"epi_num","l_win_prob":"win_prob"})
lose_table["result"] = 0
lose_table = lose_table.reset_index()
lose_table[:3]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result
0,0,1/19/2015,박준우,최현석,소유진,-1,0.5,0
1,1,1/19/2015,샘킴,홍석천,소유진,-1,0.5,0
2,2,1/26/2015,김풍,미카엘,가희,-1,0.5,0


In [196]:
match_data = win_table.append(lose_table)
match_data[:3]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result
0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1
1,1,1/19/2015,홍석천,샘킴,소유진,-1,0.5,1
2,2,1/26/2015,미카엘,김풍,가희,-1,0.5,1


In [197]:
print(len(win_table))
print(len(lose_table))
print(len(match_data))

313
313
626


In [198]:
match_data = match_data.sort_values(by = "index", ascending = True)
match_data[:5]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result
0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1
0,0,1/19/2015,박준우,최현석,소유진,-1,0.5,0
1,1,1/19/2015,샘킴,홍석천,소유진,-1,0.5,0
1,1,1/19/2015,홍석천,샘킴,소유진,-1,0.5,1
2,2,1/26/2015,미카엘,김풍,가희,-1,0.5,1


# Step 3: Additional data (age, gender, etc)

In [199]:
with open("../data/person_age_dict.pkl", 'rb') as fp:
    person_age_dict = pickle.load(fp)

In [200]:
with open("../data/id_chef_dict.pkl", 'rb') as fp:
    id_chef_dict = pickle.load(fp)

## Add chef age - guest age

In [203]:
# with open("../data/chef_age_dict.pkl", 'wb') as fp:
#     pickle.dump(chef_age_dict, fp)
    
with open("../data/chef_age_dict.pkl", 'rb') as fp:
    chef_age_dict = pickle.load(fp)

In [204]:
# with open("../data/chef_gender_dict.pkl", 'wb') as fp:
#     pickle.dump(chef_gender_dict, fp)
    
with open("../data/chef_gender_dict.pkl", 'rb') as fp:
    chef_gender_dict = pickle.load(fp)

In [205]:
with open("../data/people_df.pkl", 'rb') as fp:
    people_df = pickle.load(fp)

In [206]:
people_df[:3]

,names,gender,category,age
0,이대호,1,guest,36
1,김성균_(배우),1,guest,38
2,소미,0,guest,17


In [207]:
people_gender_dict = dict(zip(people_df.names, people_df.gender))

In [208]:
# with open("../data/people_gender_dict.pkl", 'wb') as fp:
#     pickle.dump(people_gender_dict, fp)
    
with open("../data/people_gender_dict.pkl", 'rb') as fp:
    people_gender_dict = pickle.load(fp)

In [209]:
match_data["c_age"] = match_data["chef_name"].apply(lambda x: chef_age_dict.get(x,-1))
match_data["g_age"] = match_data["guest_name"].apply(lambda x: person_age_dict.get(x,-1))
match_data["age_delta"] = abs(match_data.c_age - match_data.g_age)

In [210]:
match_data[:3]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta
0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1,46,37,9
0,0,1/19/2015,박준우,최현석,소유진,-1,0.5,0,35,37,2
1,1,1/19/2015,샘킴,홍석천,소유진,-1,0.5,0,41,37,4


In [211]:
match_data["c_gender"] = match_data["chef_name"].apply(lambda x: chef_gender_dict.get(x,-1))
match_data["g_gender"] = match_data["guest_name"].apply(lambda x: people_gender_dict.get(x,-1))
match_data["gender_delta"] = abs(match_data.c_gender - match_data.g_gender)

In [212]:
match_data[:3]

,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta,c_gender,g_gender,gender_delta
0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1,46,37,9,1,0,1
0,0,1/19/2015,박준우,최현석,소유진,-1,0.5,0,35,37,2,1,0,1
1,1,1/19/2015,샘킴,홍석천,소유진,-1,0.5,0,41,37,4,1,0,1


In [213]:
match_data = match_data.reset_index()

In [214]:
match_data[:3]

,level_0,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta,c_gender,g_gender,gender_delta
0,0,0,1/19/2015,최현석,박준우,소유진,-1,0.5,1,46,37,9,1,0,1
1,0,0,1/19/2015,박준우,최현석,소유진,-1,0.5,0,35,37,2,1,0,1
2,1,1,1/19/2015,샘킴,홍석천,소유진,-1,0.5,0,41,37,4,1,0,1


In [168]:
# Shuffle data frames
# match_data_sh = match_data.sample(frac=1).reset_index(drop=True)
# match_data_sh[:3]

In [219]:
test_df = copy.deepcopy(match_data)

In [223]:
test_df = test_df[["chef_name","against","result"]]
test_df[:3]

,chef_name,against,result
0,최현석,박준우,1
1,박준우,최현석,0
2,샘킴,홍석천,0


## Calculate individual elo pair score
* Calcualte Poong's accumulative elo score against LYB

In [220]:
# Make chef pairs
chef_pair = test_df[["chef_name","against"]].values

In [222]:
len(chef_pair)

626

In [323]:
def select_rows(chef_pair):
    a_chef, b_chef = p[0], p[1]
    mask = match_data["chef_name"] == a_chef
    a_df = match_data.loc[mask]
    mask = a_df["against"] == b_chef
    ab_df = a_df.loc[mask]
    
    return ab_df     

In [324]:
def calculate_elo(df):
    n_teams = 2
    current_elos = np.ones(shape=(n_teams)) * mean_elo
    for row in df.itertuples():
        idx = row.Index
        w_yn = row.result
        if w_yn == 0:
            w_id = 1
            l_id = 0
        else:
            w_id = 0
            l_id = 1

        # Get current elos
        w_elo_before = current_elos[w_id]
        l_elo_before = current_elos[l_id]
        w_elo_after, l_elo_after = update_elo(w_elo_before, l_elo_before)

        # Save updated elos
        if w_yn == 0:
            df.at[idx, '0_elo_before_game'] = l_elo_before
            df.at[idx, '1_elo_before_game'] = w_elo_before
            df.at[idx, '0_elo_after_game'] = l_elo_after
            df.at[idx, '1_elo_after_game'] = w_elo_after
            current_elos[w_id] = w_elo_after
            current_elos[l_id] = l_elo_after
        else:
            df.at[idx, '0_elo_before_game'] = w_elo_before
            df.at[idx, '1_elo_before_game'] = l_elo_before
            df.at[idx, '0_elo_after_game'] = w_elo_after
            df.at[idx, '1_elo_after_game'] = l_elo_after
            current_elos[w_id] = w_elo_after
            current_elos[l_id] = l_elo_after       
    return df

In [325]:
def calculate_expected_rate(df):
    before_elo_p = df["0_elo_before_game"].tolist()
    before_elo_y = df["1_elo_before_game"].tolist()
    expected_win = []
    for idx, a in enumerate(before_elo_p):
        expected_a = expected_result(a, before_elo_y[idx])
        expected_win.append(expected_a)
        
    df["expected_win_a"] = pd.Series(expected_win).values
    return df

In [326]:
def update_to_real_db(df, total_df):
    df = df[["chef_name","against","expected_win_a"]]
    p_idx_list = df.index
    p_idx = 0
    for row in total_df.itertuples():
        idx = row.Index
        if idx not in p_idx_list:
            pass
        else:
            win_pro = df.iloc[p_idx]
            win_pro_list = list(win_pro.items())
            total_df.at[idx,'against_specific_win_p'] = win_pro_list[2][1]
            p_idx += 1

In [327]:
match_data["against_specific_win_p"] = ''
for p in chef_pair:
    selected_row_df = select_rows(p)
    after_elo = calculate_elo(selected_row_df)
    after_expected_win = calculate_expected_rate(after_elo)
    update_to_real_db(after_expected_win,match_data)

In [331]:
mask = match_data["chef_name"] == '이연복'
d = match_data.loc[mask]

In [332]:
mask = d["against"] == '김풍'
b = d.loc[mask]

In [333]:
b

,level_0,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta,c_gender,g_gender,gender_delta,against_specific_win_p
75,37,37,5/25/2015,이연복,김풍,지누_(가수),6020,0.668695,1,59,47,12,1,1,0,0.5
119,59,59,8/10/2015,이연복,김풍,최화정,6063,0.726830,1,59,57,2,1,0,1,0.591076
167,83,83,11/2/2015,이연복,김풍,서장훈,6111,0.854829,0,59,44,15,1,1,0,0.661441
220,110,110,2/1/2016,이연복,김풍,타블로,6169,0.698559,1,59,38,21,1,1,0,0.545462
294,147,147,6/6/2016,이연복,김풍,김종민,6239,0.797386,1,59,39,20,1,1,0,0.626509
317,158,158,7/18/2016,이연복,김풍,존_박,6265,0.777153,1,59,30,29,1,1,0,0.688361
394,197,197,11/28/2016,이연복,김풍,정준하,6339,0.555891,1,59,47,12,1,1,0,0.73538
435,217,217,2/6/2017,이연복,김풍,김윤아,6380,0.751241,0,59,44,15,1,0,1,0.771548
463,231,231,4/3/2017,이연복,김풍,정용화,-1,0.611771,0,59,29,30,1,1,0,0.656689
501,250,250,6/12/2017,이연복,김풍,송은이,-1,0.568641,1,59,45,14,1,0,1,0.541084


In [334]:
mask = match_data["chef_name"] == '김풍'
d = match_data.loc[mask]

In [335]:
mask = d["against"] == '이연복'
b = d.loc[mask]

In [336]:
b

,level_0,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta,c_gender,g_gender,gender_delta,against_specific_win_p
74,37,37,5/25/2015,김풍,이연복,지누_(가수),6019,0.331305,0,40,47,7,1,1,0,0.5
118,59,59,8/10/2015,김풍,이연복,최화정,6064,0.273170,0,40,57,17,1,0,1,0.408924
166,83,83,11/2/2015,김풍,이연복,서장훈,6112,0.145171,1,40,44,4,1,1,0,0.338559
221,110,110,2/1/2016,김풍,이연복,타블로,-1,0.301441,0,40,38,2,1,1,0,0.454538
295,147,147,6/6/2016,김풍,이연복,김종민,6240,0.202614,0,40,39,1,1,1,0,0.373491
316,158,158,7/18/2016,김풍,이연복,존_박,6266,0.222847,0,40,30,10,1,1,0,0.311639
395,197,197,11/28/2016,김풍,이연복,정준하,6340,0.444109,0,40,47,7,1,1,0,0.26462
434,217,217,2/6/2017,김풍,이연복,김윤아,6379,0.248759,1,40,44,4,1,0,1,0.228452
462,231,231,4/3/2017,김풍,이연복,정용화,6408,0.388229,1,40,29,11,1,1,0,0.343311
500,250,250,6/12/2017,김풍,이연복,송은이,6450,0.431359,0,40,45,5,1,0,1,0.458916


In [338]:
with open("../data/id_chef_dict.pkl", 'rb') as fp:
    id_chef_dict = pickle.load(fp)

In [339]:
id_chef_dict[0]

'김풍'

In [340]:
chef_id_dict = {id_chef_dict[k] : k for k in id_chef_dict}

In [341]:
chef_id_dict["김풍"]

0

In [343]:
match_data["chef_id"] = match_data["chef_name"].apply(lambda x: chef_id_dict.get(x))
match_data["against_id"] = match_data["against"].apply(lambda x: chef_id_dict.get(x))

In [349]:
# with open("../data/ultimate_df.pkl", "wb") as fp:
#     pickle.dump(match_data, fp)

In [348]:
with open("../data/ultimate_df.pkl", "rb") as fp:
    match_data = pickle.load(fp)

# Step 4: Input ingredients

In [490]:
with open("../data/epi_ingredients_dict.pkl", "rb") as fp:
    epi_ingredients_dict = pickle.load(fp)

In [491]:
epi_ingredients_dict[6240]

['베이컨',
 '소면',
 '햄 통조림',
 '청경채',
 '양파',
 '대파',
 '다진 마늘',
 '고추장',
 '고춧가루',
 '간장',
 '물엿',
 '설탕',
 '전분']

In [511]:
ingredients = []
for k in list(epi_ingredients_dict.keys()):
    ing = epi_ingredients_dict.get(k)
    for i in ing:
        ingredients.append(i)

ing_freq = Counter(ingredients)

In [512]:
ing_df = pd.DataFrame.from_dict(ing_freq, orient = 'index').reset_index()
ing_df = ing_df.rename(columns = {'index': 'ing', 0:'cnt'})
ing_df[:3]

,ing,cnt
0,,9
1,손질가자미,1
2,카놀라유,1


In [513]:
ing_df = ing_df.sort_values(by="cnt", ascending = False).reset_index()

In [514]:
# ing_df.to_csv("../data/ing_df.csv", encoding = 'utf-8')

In [538]:
ing_df["lemm"] = ing_df["ing"]

In [539]:
ing_df[:3]

,index,ing,cnt,lemm
0,124.0,소금,415.0,소금
1,365.0,양파,343.0,양파
2,425.0,후추,335.0,후추


In [540]:
# ing_df.at[0,"lemm"] = '소금'

In [524]:
# for row in ing_df.itertuples():
#     print(row.Index)
#     print(row.ing)

In [532]:
#ing_freq.most_common(100)

In [535]:
lemm = pd.read_csv("../data/ing.txt", encoding = 'utf-8', sep = ',')

In [541]:
lemm[-3:]

,row_idx,inga,ingb
108,76,NaN,참기름
109,45,NaN,빵
110,34,NaN,돼지고기


In [542]:
for row in lemm.itertuples():
    row_idx = row.row_idx
    new_name = row.ingb
    ing_df.at[row_idx,"lemm"] = new_name
    

In [543]:
ing_df[:10]

,index,ing,cnt,lemm
0,124.0,소금,415.0,깨소금후추
1,365.0,양파,343.0,양파
2,425.0,후추,335.0,깨소금후추
3,284.0,대파,276.0,파
4,279.0,달걀,261.0,달걀
5,125.0,마늘,246.0,마늘
6,174.0,설탕,237.0,설탕
7,298.0,밀가루,217.0,밀가루
8,21.0,간장,213.0,간장
9,800.0,버터,197.0,버터


In [544]:
ing_df["index"] = pd.Series(list(range(0,len(ing_df)))).values 

In [545]:
ing_lemm_dict = dict(zip(ing_df.ing, ing_df.lemm))
ing_index_dict = dict(zip(ing_df.ing, ing_df.index))
index_ing_dict = dict(zip(ing_df.index, ing_df.ing))
ing_count_dict = dict(zip(ing_df.ing, ing_df.cnt))

In [555]:
lemm_ing_df = ing_df[["lemm", "cnt"]]

In [556]:
lemm_ing_df = lemm_ing_df.groupby("lemm")["cnt"].apply(sum).reset_index()

In [557]:
lemm_ing_df = lemm_ing_df.sort_values(by="cnt", ascending = False)

In [558]:
lemm_ing_df["lem_ing_id"] = pd.Series(list(range(0,len(lemm_ing_df)))).values

In [561]:
lemm_ing_df[:20]

,lemm,cnt,lem_ing_id
144,깨소금후추,808.0,0
352,마늘,429.0,1
793,양파,344.0,2
1176,파,310.0,3
203,달걀,261.0,4
642,설탕,237.0,5
81,고추,220.0,6
467,밀가루,217.0,7
13,간장,215.0,8
842,오일,202.0,9


In [564]:
#100번 이상 나온 건 버리기
mask = lemm_ing_df["cnt"] <= 100
key_ing_df = lemm_ing_df.loc[mask]

key_leming_lemid_dict = dict(zip(key_ing_df.lemm, key_ing_df.lem_ing_id))

In [565]:
from collections import defaultdict

epi_ingredients_new_dict = defaultdict(list)

for k in list(epi_ingredients_dict.keys()):
    ing = epi_ingredients_dict.get(k)
    for i in ing:
        lem_ing = ing_lemm_dict.get(i, -1)
        if lem_ing == -1:
            pass
        else:
            if lem_ing in key_leming_lemid_dict.keys():
                epi_ingredients_new_dict[k].append(lem_ing)
            else:
                pass

In [567]:
epi_ingredients_new_dict[6240]

['베이컨', '소면', '햄 통조림', '청경채', '고추가루', '물엿']

In [566]:
epi_ingredients_dict[6240]

['베이컨',
 '소면',
 '햄 통조림',
 '청경채',
 '양파',
 '대파',
 '다진 마늘',
 '고추장',
 '고춧가루',
 '간장',
 '물엿',
 '설탕',
 '전분']

In [913]:
def chef_ingredients(chef_name):
    mask = match_data["chef_name"] == chef_name
    poong = match_data.loc[mask]
    if len(poong) <= 5:
        pass
    else:
        poong = poong[["chef_name","against","epi_num","result"]].reset_index()
        poong["ingredients"] = poong["epi_num"].apply(lambda x: epi_ingredients_new_dict.get(x, [-1]))
        total_poong_ing = poong["ingredients"].tolist()
        poong_ing = [x for y in total_poong_ing for x in y ]

        poong_ing = list(set(poong_ing))
        poong_ing = [x for x in poong_ing if x != -1]
        poong_ing_id_dict = dict(zip(poong_ing, list(range(0,len(poong_ing)))))

        X, Y = generate_X_Y(poong, poong_ing,poong_ing_id_dict)
        y_categorical = np_utils.to_categorical(Y, 2)

        model = create_model(X)

        ratio = 0.80
        x_train = np.array(X[:int((len(X)*ratio))])
        x_test = np.array(X[len(x_train):])
        y_train = np.array(y_categorical[:int((len(y_categorical)*ratio))])
        y_test = np.array(y_categorical[len(y_train):])


        model.fit(x_train, y_train,batch_size=32, epochs=4,verbose=0,validation_data=(x_test, y_test))

        prediction = model.predict(x_test)
        predicted_classes = model.predict_classes(x_test)
        accuracy_ = categorical_accuracy(y_test,predicted_classes)

    #return accuracy

In [914]:
for chef in list(chef_age_dict.keys()):
    print(chef)
    accuracy = chef_ingredients(chef)
    print("-------------")

백승욱
-------------
이찬오
Accuracy:  0.2
-------------
유현수
Accuracy:  0.6
-------------
정형돈
-------------
박건영
-------------
홍석천
Accuracy:  0.4
-------------
이상민
-------------
미카엘
Accuracy:  0.38461538461538464
-------------
여경래
-------------
김풍
Accuracy:  0.6875
-------------
모토카와 아쓰시
-------------
이재훈
Accuracy:  0.3333333333333333
-------------
이연복
Accuracy:  0.5714285714285714
-------------
오세득
Accuracy:  0.4444444444444444
-------------
최형진
-------------
최현석
Accuracy:  0.6363636363636364
-------------
유창준
-------------
박리혜
-------------
정호영
Accuracy:  0.4
-------------
김민준
-------------
맹기용
-------------
정창욱
Accuracy:  0.3333333333333333
-------------
이원일
Accuracy:  0.42857142857142855
-------------
박준우
Accuracy:  0.6666666666666666
-------------
정지선
-------------
김성주
-------------
샘킴
Accuracy:  0.3125
-------------
주배안
-------------
고든 램지
-------------
레이먼 킴
Accuracy:  0.2
-------------
한상훈
-------------


In [846]:
# Test with Poong, test with LYB
mask = match_data["chef_name"] == '최현석'
poong = match_data.loc[mask]

In [847]:
len(poong)

55

In [848]:
poong[:3]

,level_0,index,date,chef_name,against,guest_name,epi_num,win_prob,result,c_age,g_age,age_delta,c_gender,g_gender,gender_delta,against_specific_win_p,chef_id,against_id
0,0,0,1/19/2015,최현석,박준우,소유진,-1,0.500000,1,46,37,9,1,0,1,0.5,4,12
13,6,6,2/9/2015,최현석,정창욱,허경환,-1,0.500000,1,46,37,9,1,1,0,0.5,4,14
23,11,11,2/23/2015,최현석,김풍,문희준,-1,0.676302,1,46,40,6,1,1,0,0.5,4,0


In [849]:
poong = poong[["chef_name","against","epi_num","result"]].reset_index()

In [850]:
poong["ingredients"] = poong["epi_num"].apply(lambda x: epi_ingredients_new_dict.get(x, [-1]))

In [851]:
poong[-10:]

,index,chef_name,against,epi_num,result,ingredients
45,391,최현석,샘킴,6336,0,"[조개, 시금치, 관자, 전복, 새우, 와인]"
46,398,최현석,백승욱,6343,0,"[갈치, 명란젓, 자숙새우, 시금치, 아보카도, 브로콜리, 시금치딥, 황도, 블랙올..."
47,404,최현석,미카엘,6354,1,"[서대, 새우, 돼지고기 목심, 두릅, 삶은 죽순, 오렌지주스, 아로니아즙]"
48,416,최현석,김풍,6366,1,"[해물볶음고추장, 굴소스, 양념게장, 새우젓, 깻잎, 상추]"
49,424,최현석,레이먼 킴,6373,0,"[파스타면, 아스파라거스, 토마토, 양상추, 관자, 베이컨, 오레가노, 액상조미료]"
50,431,최현석,유현수,6375,1,"[새우살, 대게살, 날치알, 딸기주스, 우유]"
51,438,최현석,이재훈,6383,1,"[소고기등심, 치킨스톡, 멸치액젓, 게맛살, 사과, 치즈, 쌀]"
52,448,최현석,샘킴,6397,0,"[소갈빗살, 두부, 칼국수면, 인스턴트 꼬리곰탕, 조미료, 고추가루]"
53,456,최현석,오세득,6405,0,"[빵, 새우젓, 멸치액젓, 알배추, 굴, 베이컨, 허니머스터드, 파르메산 치즈가루,..."
54,467,최현석,미카엘,6411,0,"[전복, 새우, 우유, 카망베르치즈, 치즈, 취장아찌, 양념명란젓, 톳가루]"


In [852]:
total_poong_ing = poong["ingredients"].tolist()

In [853]:
poong_ing = [x for y in total_poong_ing for x in y ]

In [854]:
poong_ing = list(set(poong_ing))

In [855]:
poong_ing = list(set(poong_ing))
poong_ing = [x for x in poong_ing if x != -1]

In [856]:
poong_ing = list(set(poong_ing))
poong_ing = [x for x in poong_ing if x != -1]
poong_ing_id_dict = dict(zip(poong_ing, list(range(0,len(poong_ing)))))

In [857]:
#poong_ing_id_dict["인절미"]

In [858]:
len(poong_ing)

246

In [893]:
def generate_X_Y(poong, poong_ing,poong_ing_id_dict):
    poong_mat = np.zeros([len(poong),len(poong_ing)+1])
    for row in poong.itertuples():
        idx = row.Index
        #print("idx:",idx)
        ingre = row.ingredients
        result = row.result
        #print("result:",result)
        if ingre == [-1]:
            poong_mat[idx,len(poong_ing)] = result
            #print(poong_mat[idx])
        else:
            for i in ingre:
                #print("ingre:", i)
                poong_mat[idx,poong_ing_id_dict.get(i)] += 1
            poong_mat[idx,len(poong_ing)] = result
    X = poong_mat[:, 0:poong_mat.shape[1]-1]
    Y = poong_mat[:,poong_mat.shape[1]-1]
    
    return X, Y

In [859]:
poong_mat = np.zeros([len(poong),len(poong_ing)+1])

In [860]:
poong_mat.shape

(55, 247)

In [861]:
for row in poong.itertuples():
    idx = row.Index
    #print("idx:",idx)
    ingre = row.ingredients
    result = row.result
    #print("result:",result)
    if ingre == [-1]:
        poong_mat[idx,len(poong_ing)] = result
        #print(poong_mat[idx])
    else:
        for i in ingre:
            poong_mat[idx,poong_ing_id_dict.get(i)] += 1
        poong_mat[idx,len(poong_ing)] = result
        #print(poong_mat[idx])
    #print("---------------------")

In [862]:
# Split into input(X) and output(Y) variables
X = poong_mat[:, 0:poong_mat.shape[1]-1]
Y = poong_mat[:,poong_mat.shape[1]-1]

In [863]:
y_categorical = np_utils.to_categorical(Y, 2)
print(y_categorical.shape)
print(y_categorical[0].shape)
print(y_categorical[:3])

(55, 2)
(2,)
[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [864]:
X.shape

(55, 246)

In [865]:
X[0].shape

(246,)

In [866]:
from sklearn.preprocessing import LabelEncoder
# Encode class variables as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [867]:
print(Y)
print(encoded_Y)

[ 1.  1.  1.  1.  1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  1.  0.
  1.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  1.  0.  1.  0.
  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  1.  1.  0.  1.  1.  0.  0.
  0.]
[1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0
 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0]


In [868]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [869]:
import tensorflow as tf

In [870]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [884]:
def create_model(X):
    model = Sequential()
    model.add(Dense(60, input_dim = X[0].shape[0], kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model
    


In [871]:
# Create model
model = Sequential()
model.add(Dense(60, input_dim = X[0].shape[0], kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))

In [872]:
# Compile
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam')
model.compile(loss='categorical_crossentropy', optimizer='adam')
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [873]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model, epochs=100, batch_size=5, verbose=0)

In [874]:
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [875]:
#results = cross_val_score(estimator, X, encoded_Y, cv=kfold)

In [876]:
#print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [877]:
ratio = 0.80
x_train = np.array(X[:int((len(X)*ratio))])
x_test = np.array(X[len(x_train):])
y_train = np.array(y_categorical[:int((len(y_categorical)*ratio))])
y_test = np.array(y_categorical[len(y_train):])

In [878]:
model.fit(x_train, y_train,
          batch_size=32, epochs=4,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 44 samples, validate on 11 samples
Epoch 1/4
44/44 [==============================] - 1s 13ms/step - loss: 0.6929 - val_loss: 0.6915
Epoch 2/4
44/44 [==============================] - 0s 189us/step - loss: 0.6884 - val_loss: 0.6915
Epoch 3/4
44/44 [==============================] - 0s 216us/step - loss: 0.6845 - val_loss: 0.6913
Epoch 4/4
44/44 [==============================] - 0s 218us/step - loss: 0.6807 - val_loss: 0.6909


In [879]:
score = model.evaluate(x_test, y_test, verbose = 1)
score

11/11 [==============================] - 0s 144us/step


0.69094091653823853

In [880]:
prediction = model.predict(x_test)
predicted_classes = model.predict_classes(x_test)

def categorical_accuracy(y_true, y_pred):
    accu = 0
    for idx, y in enumerate(y_true):
        y_t = 0
        if y[0] == 1:
            pass
        else:
            y_t += 1
            
        if y_t == y_pred[idx]:
            accu += 1
    accuracy = accu / len(y_true)
    print("Accuracy: ", accuracy)
    return accuracy

accuracy = categorical_accuracy(y_test,predicted_classes)

Accuracy:  0.5454545454545454


In [881]:
prediction

array([[ 0.51931721,  0.49867606],
       [ 0.51641452,  0.49760753],
       [ 0.50958711,  0.50734764],
       [ 0.50554132,  0.50316572],
       [ 0.49938005,  0.50849146],
       [ 0.50265163,  0.51532006],
       [ 0.50429767,  0.50575972],
       [ 0.51081771,  0.5071609 ],
       [ 0.49922368,  0.49977478],
       [ 0.50551641,  0.52333009],
       [ 0.51534575,  0.50542682]], dtype=float32)

In [882]:
y_test

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

# Step 5: Prepare x(input) and y(output)

## Y

In [469]:
y_ = match_data["result"].tolist()

In [470]:
y_categorical = np_utils.to_categorical(y_, 2)
print(y_categorical.shape)
print(y_categorical[0].shape)
print(y_categorical[:3])

(626, 2)
(2,)
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]


## X

In [471]:
#x_df = match_data[["chef_id","against_id","win_prob","age_delta","gender_delta","against_specific_win_p"]]
x_df = match_data[["chef_id","against_id","win_prob","age_delta","gender_delta","against_specific_win_p"]]
x_df[:3]

,chef_id,against_id,win_prob,age_delta,gender_delta,against_specific_win_p
0,4,12,0.5,9,1,0.5
1,12,4,0.5,2,1,0.5
2,1,8,0.5,4,1,0.5


In [472]:
x_ = x_df.as_matrix()
x_[:3]

array([[4, 12, 0.5, 9, 1, 0.5],
       [12, 4, 0.5, 2, 1, 0.5],
       [1, 8, 0.5, 4, 1, 0.5]], dtype=object)

In [473]:
print(x_.shape)
print(x_[0].shape)

(626, 6)
(6,)


In [474]:
shp = x_[0].shape[0]

#### Divide the dataset into trian & test dataset

In [475]:
ratio = 0.80
x_train = np.array(x_[:int((len(x_)*ratio))])
x_test = np.array(x_[len(x_train):])
y_train = np.array(y_categorical[:int((len(y_categorical)*ratio))])
y_test = np.array(y_categorical[len(y_train):])

In [476]:
print(len(x_train))
print(len(x_test))

500
126


In [477]:
y_test.shape

(126, 2)

In [478]:
y_test[0].shape

(2,)

# Step 6: Define model architecture
The simplest model is defined in the Sequential class which is a linear stack of Layers.

In [479]:
# Declare Sequential model
model = Sequential()

In [480]:
# Next, declare input layer. Input shape parameter is the shape of 1 sample.
model.add(Dense(500, input_shape=(shp,)))
model.add(Activation('relu')) 
# An "activation" is just a non-linear function applied to the output of the layer above.

# If I print, the current shape of the model output, it will return me this.
print(model.output_shape)

(None, 500)


In [481]:
# Now add more layers.
# We want to add Dropout layer to regularize the model in order to prevent overfitting.
model.add(Dropout(0.2))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2)) # Corresponds to the final output size of 2.
model.add(Activation('softmax'))

This special "softmax" activation among other things, ensures the output is a valid probaility distribution, that its values are all non-negative and sum to 1.

# Step 7: Compile model
Cannot believe it is almost done already. The hard part is already over and I will just compile the model.

In [482]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Step 8: Fit model on training data

In [483]:
model.fit(x_train, y_train,
          batch_size=32, epochs=4,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 500 samples, validate on 126 samples
Epoch 1/4
500/500 [==============================] - 1s 1ms/step - loss: 1.0055 - val_loss: 0.7132
Epoch 2/4
500/500 [==============================] - 0s 220us/step - loss: 0.7675 - val_loss: 0.7132
Epoch 3/4
500/500 [==============================] - 0s 185us/step - loss: 0.7632 - val_loss: 0.7381
Epoch 4/4
500/500 [==============================] - 0s 203us/step - loss: 0.7601 - val_loss: 0.8101


### That was easy. huh?

## Step 6: Evaluate model on test data

In [484]:
score = model.evaluate(x_test, y_test, verbose = 1)
score

126/126 [==============================] - 0s 98us/step


0.81008685297436189

Score is the mean of the loss for each test sample. 

### Accuracy
The predict_classes function outputs the highest probability class according to the trained classifier for each input example.

In [485]:
prediction = model.predict(x_test)
predicted_classes = model.predict_classes(x_test)

In [486]:
prediction = model.predict(x_test)
predicted_classes = model.predict_classes(x_test)

def categorical_accuracy(y_true, y_pred):
    accu = 0
    for idx, y in enumerate(y_true):
        y_t = 0
        if y[0] == 1:
            pass
        else:
            y_t += 1
            
        if y_t == y_pred[idx]:
            accu += 1
    accuracy = accu / len(y_true)
    print("Accuracy: ", accuracy)
    return accuracy

accuracy = categorical_accuracy(y_test,predicted_classes)

In [487]:
accuracy = categorical_accuracy(y_test,predicted_classes)

Accuracy:  0.47619047619047616


In [488]:
y_categorical[:10]

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [489]:
prediction[:10]

array([[ 0.59442014,  0.40557989],
       [ 0.64226657,  0.3577334 ],
       [ 0.65541857,  0.34458145],
       [ 0.59892881,  0.40107116],
       [ 0.54136086,  0.45863917],
       [ 0.65934926,  0.34065074],
       [ 0.60531616,  0.39468381],
       [ 0.57603872,  0.42396122],
       [ 0.8457908 ,  0.15420923],
       [ 0.63563502,  0.36436501]], dtype=float32)